# 🧪 Complete Strix System Services Testing

Notebook completo para probar todos los servicios del sistema Strix incluyendo:
- ✅ Servicios Core (Domain, Event, Portfolio, S3, Vehicle)
- 🔐 Servicios Security (User, Access Control, Health Check) 
- 🌐 API Endpoints
- 🔗 Tests de Integración

**Ubicación:** `strix_complete_services_test.ipynb`

---

## 🚀 1. Setup e Inicialización

In [ ]:
# Setup inicial del entorno
import sys
import os
from pathlib import Path
from datetime import datetime, timedelta
import json
import traceback
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Configurar paths del proyecto
project_root = Path().absolute()
if 'strix-system' not in str(project_root):
    # Buscar el directorio del proyecto
    current = Path().absolute()
    while current.name != 'strix-system' and current != current.parent:
        current = current.parent
    if current.name == 'strix-system':
        project_root = current

sys.path.insert(0, str(project_root))

print(f"📁 Proyecto root: {project_root}")
print(f"🐍 Python version: {sys.version}")
print(f"🕐 Inicio del testing: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 60)

In [ ]:
# Variables globales para el testing
TEST_SUFFIX = datetime.now().strftime("%H%M%S")
TEST_RESULTS = {}
CREATED_TEST_DATA = []

print(f"🏷️ Sufijo de test único: {TEST_SUFFIX}")
print(f"📊 Resultados se almacenarán en: TEST_RESULTS")
print(f"🗑️ Datos de prueba se trackearán en: CREATED_TEST_DATA")

## 🔍 2. Verificación de Imports y Dependencias

In [ ]:
def test_imports():
    """Verifica que todos los imports funcionen correctamente"""
    print("🔍 Verificando imports...")
    import_results = {}
    
    imports_to_test = [
        # Core Infrastructure
        ("Postgres", "core.infraestructure.db.postgres", "Postgres"),
        ("S3Client", "core.infraestructure.aws.s3", "S3Client"),
        ("SQLAlchemyUnitOfWork", "core.infraestructure.uow.sqlalchemy_uow", "SQLAlchemyUnitOfWork"),
        
        # Core Models
        ("Domain Model", "core.models.domain", "Domain"),
        ("Event Model", "core.models.event", "Event"),
        ("Portfolio Model", "core.models.portfolio", "Portfolio"),
        
        # Core Services
        ("DomainService", "core.services.domain_service", "DomainService"),
        ("EventService", "core.services.event_service", "EventService"),
        ("PortfolioService", "core.services.portfolio_service", "PortfolioService"),
        ("S3EventService", "core.services.s3_event_service", "S3EventService"),
        ("VehicleService", "core.services.vehicle_service", "VehicleService"),
        ("EventDomainService", "core.services.event_domain_service", "EventDomainService"),
        
        # Security Services
        ("UserService", "security.services.user_service", "UserService"),
        ("AccessControlService", "security.services.access_control_service", "AccessControlService"),
        ("HealthCheck", "security.services.health_check", "health_check"),
        
        # API
        ("FastAPI Main", "api.main", "app")
    ]
    
    for name, module, class_name in imports_to_test:
        try:
            exec(f"from {module} import {class_name}")
            import_results[name] = {"status": "✅ OK", "error": None}
            print(f"✅ {name}")
        except Exception as e:
            import_results[name] = {"status": "❌ FAIL", "error": str(e)}
            print(f"❌ {name}: {e}")
    
    return import_results

# Ejecutar test de imports
import_test_results = test_imports()
TEST_RESULTS['imports'] = import_test_results

successful_imports = sum(1 for r in import_test_results.values() if "✅" in r["status"])
total_imports = len(import_test_results)
print(f"\n📊 Resumen imports: {successful_imports}/{total_imports} exitosos")

## 🏥 3. Health Check General del Sistema

In [ ]:
def system_health_check():
    """Health check completo del sistema"""
    print("🏥 Ejecutando Health Check del Sistema...")
    health_results = {}
    
    # 1. PostgreSQL
    try:
        from core.infraestructure.db.postgres import Postgres
        from sqlalchemy import text
        
        db = Postgres()
        engine = db.connPostgres()
        
        if engine:
            with engine.connect() as conn:
                result = conn.execute(text("SELECT 1, NOW()"))
                row = result.fetchone()
                health_results['postgresql'] = {
                    "status": "✅ OK", 
                    "details": f"Connected at {row[1]}"
                }
                print("✅ PostgreSQL: Conectado")
        else:
            health_results['postgresql'] = {"status": "❌ FAIL", "details": "No connection"}
            print("❌ PostgreSQL: Sin conexión")
    except Exception as e:
        health_results['postgresql'] = {"status": "❌ FAIL", "details": str(e)}
        print(f"❌ PostgreSQL: {e}")
    
    # 2. S3
    try:
        from core.infraestructure.aws.s3 import S3Client
        
        s3_client = S3Client()
        client = s3_client.conn_s3()
        
        if client:
            config = s3_client.config
            health_results['s3'] = {
                "status": "✅ OK", 
                "details": f"Bucket: {config.get('bucket_name', 'N/A')}"
            }
            print("✅ S3: Conectado")
        else:
            health_results['s3'] = {"status": "❌ FAIL", "details": "No connection"}
            print("❌ S3: Sin conexión")
    except Exception as e:
        health_results['s3'] = {"status": "❌ FAIL", "details": str(e)}
        print(f"❌ S3: {e}")
    
    # 3. Security Health Check
    try:
        from security.services.health_check import health_check
        
        security_health = health_check()
        health_results['security_service'] = {
            "status": "✅ OK", 
            "details": security_health
        }
        print(f"✅ Security Health Check: {security_health}")
    except Exception as e:
        health_results['security_service'] = {"status": "❌ FAIL", "details": str(e)}
        print(f"❌ Security Health Check: {e}")
    
    return health_results

# Ejecutar health check
health_results = system_health_check()
TEST_RESULTS['health_check'] = health_results

print("\n📋 Resumen Health Check:")
for service, result in health_results.items():
    print(f"  {service}: {result['status']}")

## 🧱 4. Testing de Servicios Core

In [ ]:
def test_core_services():
    """Tests completos de servicios core"""
    print("🧱 Testing Servicios Core...")
    core_results = {}
    
    # 1. DomainService
    print("\n📍 1. DOMAIN SERVICE")
    try:
        from core.services.domain_service import DomainService
        
        domain_service = DomainService()
        print("✅ DomainService instanciado")
        
        # Test crear domain
        test_domain_name = f"TEST_DOMAIN_{TEST_SUFFIX}"
        created_domain = domain_service.create_domain(
            test_domain_name, 
            f"mrn:thing:test_{TEST_SUFFIX}", 
            f"mrn:account:test_{TEST_SUFFIX}"
        )
        
        if created_domain:
            domain_id = created_domain['id']
            CREATED_TEST_DATA.append({'type': 'domain', 'id': domain_id})
            print(f"✅ Domain creado: ID {domain_id}")
            
            # Test leer domain
            retrieved_domain = domain_service.get_domain_by_id(domain_id)
            if retrieved_domain and retrieved_domain['domain'] == test_domain_name:
                print("✅ Domain recuperado correctamente")
                
                # Test buscar por nombre
                by_name = domain_service.get_domain_by_name(test_domain_name)
                if by_name:
                    print("✅ Búsqueda por nombre exitosa")
                    core_results['domain_service'] = {"status": "✅ OK", "domain_id": domain_id}
                else:
                    print("❌ Búsqueda por nombre falló")
                    core_results['domain_service'] = {"status": "⚠️ PARTIAL", "issue": "Search by name failed"}
            else:
                print("❌ Domain no recuperado correctamente")
                core_results['domain_service'] = {"status": "❌ FAIL", "issue": "Retrieval failed"}
        else:
            print("❌ No se pudo crear domain")
            core_results['domain_service'] = {"status": "❌ FAIL", "issue": "Creation failed"}
            
    except Exception as e:
        print(f"❌ Error en DomainService: {e}")
        core_results['domain_service'] = {"status": "❌ FAIL", "error": str(e)}
    
    # 2. PortfolioService
    print("\n📂 2. PORTFOLIO SERVICE")
    try:
        from core.services.portfolio_service import PortfolioService
        
        portfolio_service = PortfolioService()
        print("✅ PortfolioService instanciado")
        
        # Obtener portfolios existentes
        existing_portfolios = portfolio_service.get_all_portfolios()
        print(f"✅ Portfolios existentes: {len(existing_portfolios) if existing_portfolios else 0}")
        
        # Test crear portfolio (solo si tenemos un domain)
        if 'domain_service' in core_results and core_results['domain_service'].get('domain_id'):
            domain_id = core_results['domain_service']['domain_id']
            test_portfolio_name = f"TEST_PORTFOLIO_{TEST_SUFFIX}"
            
            portfolio_id = portfolio_service.create_portfolio_with_domains(
                test_portfolio_name, [domain_id]
            )
            
            if portfolio_id:
                CREATED_TEST_DATA.append({'type': 'portfolio', 'id': portfolio_id})
                print(f"✅ Portfolio creado: ID {portfolio_id}")
                
                # Verificar portfolio
                retrieved_portfolio = portfolio_service.get_portfolio_by_id(portfolio_id)
                if retrieved_portfolio:
                    print("✅ Portfolio recuperado correctamente")
                    core_results['portfolio_service'] = {"status": "✅ OK", "portfolio_id": portfolio_id}
                else:
                    core_results['portfolio_service'] = {"status": "❌ FAIL", "issue": "Retrieval failed"}
            else:
                core_results['portfolio_service'] = {"status": "❌ FAIL", "issue": "Creation failed"}
        else:
            print("⚠️ Sin domain disponible para portfolio")
            core_results['portfolio_service'] = {"status": "⚠️ SKIPPED", "reason": "No domain available"}
            
    except Exception as e:
        print(f"❌ Error en PortfolioService: {e}")
        core_results['portfolio_service'] = {"status": "❌ FAIL", "error": str(e)}
    
    # 3. EventService
    print("\n📊 3. EVENT SERVICE")
    try:
        from core.infraestructure.uow.sqlalchemy_uow import SQLAlchemyUnitOfWork
        from core.services.event_service import EventService
        
        uow = SQLAlchemyUnitOfWork()
        event_service = EventService(uow)
        print("✅ EventService instanciado")
        
        # Test solo si tenemos domain
        if 'domain_service' in core_results and core_results['domain_service'].get('domain_id'):
            domain_id = core_results['domain_service']['domain_id']
            
            # Test crear evento
            timestamp = int(datetime.now().timestamp() * 1000)
            created_event = event_service.create_event(
                domain_id, -34.6118, -58.3960, 50, "test_position", timestamp, 100, 90
            )
            
            if created_event:
                event_id = created_event['id']
                CREATED_TEST_DATA.append({'type': 'event', 'id': event_id})
                print(f"✅ Event creado: ID {event_id}")
                
                # Test recuperar eventos por domain
                events_by_domain = event_service.get_events_by_domain(domain_id)
                if events_by_domain and len(events_by_domain) > 0:
                    print(f"✅ Eventos recuperados: {len(events_by_domain)}")
                    core_results['event_service'] = {"status": "✅ OK", "event_id": event_id}
                else:
                    core_results['event_service'] = {"status": "⚠️ PARTIAL", "issue": "Event retrieval failed"}
            else:
                core_results['event_service'] = {"status": "❌ FAIL", "issue": "Event creation failed"}
        else:
            print("⚠️ Sin domain disponible para events")
            core_results['event_service'] = {"status": "⚠️ SKIPPED", "reason": "No domain available"}
            
    except Exception as e:
        print(f"❌ Error en EventService: {e}")
        core_results['event_service'] = {"status": "❌ FAIL", "error": str(e)}
    
    # 4. S3EventService
    print("\n☁️ 4. S3 EVENT SERVICE")
    try:
        from core.services.s3_event_service import S3EventService
        
        s3_event_service = S3EventService()
        print("✅ S3EventService instanciado")
        
        # Test de instanciación únicamente (datos reales requerirían archivos S3)
        if hasattr(s3_event_service, 's3_client'):
            print("✅ S3Client configurado")
            core_results['s3_event_service'] = {"status": "✅ OK", "note": "Instance created, real S3 data not tested"}
        else:
            core_results['s3_event_service'] = {"status": "⚠️ PARTIAL", "issue": "S3Client not found"}
            
    except Exception as e:
        print(f"❌ Error en S3EventService: {e}")
        core_results['s3_event_service'] = {"status": "❌ FAIL", "error": str(e)}
    
    return core_results

# Ejecutar tests de servicios core
core_test_results = test_core_services()
TEST_RESULTS['core_services'] = core_test_results

print("\n📋 Resumen Core Services:")
for service, result in core_test_results.items():
    print(f"  {service}: {result['status']}")

## 🔐 5. Testing de Servicios Security

In [ ]:
def test_security_services():
    """Tests completos de servicios de security"""
    print("🔐 Testing Servicios Security...")
    security_results = {}
    
    # 1. UserService
    print("\n👤 1. USER SERVICE")
    try:
        from security.services.user_service import UserService
        
        user_service = UserService()
        print("✅ UserService instanciado")
        
        # Test crear usuario
        test_username = f"test_user_{TEST_SUFFIX}"
        test_email = f"test_{TEST_SUFFIX}@example.com"
        
        created_user = user_service.register_user(
            username=test_username,
            email=test_email,
            password="test_password_123",
            full_name=f"Test User {TEST_SUFFIX}"
        )
        
        if created_user:
            user_id = created_user.id
            CREATED_TEST_DATA.append({'type': 'user', 'id': user_id})
            print(f"✅ User creado: ID {user_id}, Username: {test_username}")
            
            # Test recuperar usuario
            retrieved_user = user_service.get_user(test_username)
            if retrieved_user and retrieved_user.username == test_username:
                print("✅ User recuperado correctamente")
                
                # Test autenticación
                auth_user = user_service.authenticate_user(test_username, "test_password_123")
                if auth_user:
                    print("✅ Autenticación exitosa")
                    security_results['user_service'] = {"status": "✅ OK", "user_id": user_id}
                else:
                    print("❌ Autenticación falló")
                    security_results['user_service'] = {"status": "⚠️ PARTIAL", "issue": "Authentication failed"}
            else:
                print("❌ User no recuperado")
                security_results['user_service'] = {"status": "❌ FAIL", "issue": "User retrieval failed"}
        else:
            print("❌ No se pudo crear user")
            security_results['user_service'] = {"status": "❌ FAIL", "issue": "User creation failed"}
            
    except Exception as e:
        print(f"❌ Error en UserService: {e}")
        security_results['user_service'] = {"status": "❌ FAIL", "error": str(e)}
    
    # 2. AccessControlService
    print("\n🛡️ 2. ACCESS CONTROL SERVICE")
    try:
        from security.services.access_control_service import AccessControlService
        
        access_service = AccessControlService()
        print("✅ AccessControlService instanciado")
        
        # Test crear permission
        test_permission_name = f"test_permission_{TEST_SUFFIX}"
        created_permission = access_service.create_permission(
            name=test_permission_name,
            description=f"Test permission created at {datetime.now()}"
        )
        
        if created_permission:
            permission_id = created_permission.id
            CREATED_TEST_DATA.append({'type': 'permission', 'id': permission_id})
            print(f"✅ Permission creado: ID {permission_id}")
            
            # Test crear role
            test_role_name = f"test_role_{TEST_SUFFIX}"
            created_role = access_service.create_role(
                name=test_role_name,
                description=f"Test role created at {datetime.now()}"
            )
            
            if created_role:
                role_id = created_role.id
                CREATED_TEST_DATA.append({'type': 'role', 'id': role_id})
                print(f"✅ Role creado: ID {role_id}")
                
                # Test asignar permission a role
                assigned_permission = access_service.assign_permission_to_role(role_id, permission_id)
                if assigned_permission:
                    print("✅ Permission asignado a role")
                    
                    # Test asignar role a user (si tenemos user)
                    if 'user_service' in security_results and security_results['user_service'].get('user_id'):
                        user_id = security_results['user_service']['user_id']
                        assigned_role = access_service.assign_role_to_user(user_id, role_id)
                        if assigned_role:
                            print("✅ Role asignado a user")
                            security_results['access_control_service'] = {"status": "✅ OK", "role_id": role_id, "permission_id": permission_id}
                        else:
                            security_results['access_control_service'] = {"status": "⚠️ PARTIAL", "issue": "Role assignment failed"}
                    else:
                        print("⚠️ Sin user para asignar role")
                        security_results['access_control_service'] = {"status": "⚠️ PARTIAL", "note": "No user available for role assignment"}
                else:
                    security_results['access_control_service'] = {"status": "❌ FAIL", "issue": "Permission assignment failed"}
            else:
                security_results['access_control_service'] = {"status": "❌ FAIL", "issue": "Role creation failed"}
        else:
            security_results['access_control_service'] = {"status": "❌ FAIL", "issue": "Permission creation failed"}
            
    except Exception as e:
        print(f"❌ Error en AccessControlService: {e}")
        security_results['access_control_service'] = {"status": "❌ FAIL", "error": str(e)}
    
    return security_results

# Ejecutar tests de servicios security
security_test_results = test_security_services()
TEST_RESULTS['security_services'] = security_test_results

print("\n📋 Resumen Security Services:")
for service, result in security_test_results.items():
    print(f"  {service}: {result['status']}")

## 🌐 6. Testing de API Endpoints

In [ ]:
def test_api_endpoints():
    """Tests de endpoints de la API usando TestClient"""
    print("🌐 Testing API Endpoints...")
    api_results = {}
    
    try:
        from fastapi.testclient import TestClient
        from api.main import app
        
        client = TestClient(app)
        print("✅ TestClient inicializado")
        
        # 1. Health Check
        print("\n🏥 1. HEALTH ENDPOINT")
        try:
            response = client.get("/health")
            if response.status_code == 200:
                health_data = response.json()
                print(f"✅ Health endpoint OK: {health_data}")
                api_results['health'] = {"status": "✅ OK", "data": health_data}
            else:
                print(f"❌ Health endpoint falló: {response.status_code}")
                api_results['health'] = {"status": "❌ FAIL", "status_code": response.status_code}
        except Exception as e:
            print(f"❌ Error en health endpoint: {e}")
            api_results['health'] = {"status": "❌ FAIL", "error": str(e)}
        
        # 2. Users Endpoints
        print("\n👤 2. USERS ENDPOINTS")
        try:
            # Test crear user via API
            test_user_data = {
                "username": f"api_user_{TEST_SUFFIX}",
                "email": f"api_{TEST_SUFFIX}@example.com",
                "password": "api_test_123",
                "full_name": f"API Test User {TEST_SUFFIX}"
            }
            
            response = client.post("/users/", json=test_user_data)
            if response.status_code in [200, 201]:
                user_data = response.json()
                print(f"✅ User creado via API: {user_data['username']}")
                
                # Test obtener user
                get_response = client.get(f"/users/{user_data['username']}")
                if get_response.status_code == 200:
                    retrieved_user = get_response.json()
                    print(f"✅ User recuperado via API: {retrieved_user['username']}")
                    api_results['users'] = {"status": "✅ OK", "user": retrieved_user['username']}
                else:
                    api_results['users'] = {"status": "⚠️ PARTIAL", "issue": "Get user failed"}
            elif response.status_code == 400:
                print("⚠️ User ya existe o error de validación")
                api_results['users'] = {"status": "⚠️ PARTIAL", "note": "User might already exist"}
            else:
                print(f"❌ Crear user falló: {response.status_code}")
                api_results['users'] = {"status": "❌ FAIL", "status_code": response.status_code}
        except Exception as e:
            print(f"❌ Error en users endpoints: {e}")
            api_results['users'] = {"status": "❌ FAIL", "error": str(e)}
        
        # 3. Roles Endpoints
        print("\n🛡️ 3. ROLES ENDPOINTS")
        try:
            test_role_data = {
                "id": 0,
                "name": f"api_role_{TEST_SUFFIX}",
                "description": f"API test role {TEST_SUFFIX}",
                "permissions": []
            }
            
            response = client.post("/roles/", json=test_role_data)
            if response.status_code in [200, 201]:
                role_data = response.json()
                print(f"✅ Role creado via API: {role_data['name']}")
                
                # Test obtener role
                get_response = client.get(f"/roles/{role_data['name']}")
                if get_response.status_code == 200:
                    print("✅ Role recuperado via API")
                    api_results['roles'] = {"status": "✅ OK", "role": role_data['name']}
                else:
                    api_results['roles'] = {"status": "⚠️ PARTIAL", "issue": "Get role failed"}
            elif response.status_code == 400:
                print("⚠️ Role ya existe")
                api_results['roles'] = {"status": "⚠️ PARTIAL", "note": "Role might already exist"}
            else:
                api_results['roles'] = {"status": "❌ FAIL", "status_code": response.status_code}
        except Exception as e:
            print(f"❌ Error en roles endpoints: {e}")
            api_results['roles'] = {"status": "❌ FAIL", "error": str(e)}
        
        # 4. Permissions Endpoints
        print("\n🔑 4. PERMISSIONS ENDPOINTS")
        try:
            test_permission_data = {
                "id": 0,
                "name": f"api_permission_{TEST_SUFFIX}",
                "description": f"API test permission {TEST_SUFFIX}"
            }
            
            response = client.post("/permissions/", json=test_permission_data)
            if response.status_code in [200, 201]:
                permission_data = response.json()
                print(f"✅ Permission creado via API: {permission_data['name']}")
                api_results['permissions'] = {"status": "✅ OK", "permission": permission_data['name']}
            elif response.status_code == 400:
                print("⚠️ Permission ya existe")
                api_results['permissions'] = {"status": "⚠️ PARTIAL", "note": "Permission might already exist"}
            else:
                api_results['permissions'] = {"status": "❌ FAIL", "status_code": response.status_code}
        except Exception as e:
            print(f"❌ Error en permissions endpoints: {e}")
            api_results['permissions'] = {"status": "❌ FAIL", "error": str(e)}
        
    except Exception as e:
        print(f"❌ Error general en API testing: {e}")
        api_results['general'] = {"status": "❌ FAIL", "error": str(e)}
    
    return api_results

# Ejecutar tests de API
api_test_results = test_api_endpoints()
TEST_RESULTS['api_endpoints'] = api_test_results

print("\n📋 Resumen API Endpoints:")
for endpoint, result in api_test_results.items():
    print(f"  {endpoint}: {result['status']}")

## 🔗 7. Tests de Integración Avanzados

In [ ]:
def test_integration_scenarios():
    """Tests de integración entre múltiples servicios"""
    print("🔗 Testing Escenarios de Integración...")
    integration_results = {}
    
    # 1. Flujo completo: Domain -> Portfolio -> Events
    print("\n🔄 1. FLUJO DOMAIN -> PORTFOLIO -> EVENTS")
    try:
        from core.services.domain_service import DomainService
        from core.services.portfolio_service import PortfolioService
        from core.services.event_service import EventService
        from core.infraestructure.uow.sqlalchemy_uow import SQLAlchemyUnitOfWork
        
        # Crear domain
        domain_service = DomainService()
        test_domain = domain_service.create_domain(
            f"INTEGRATION_DOMAIN_{TEST_SUFFIX}",
            f"mrn:thing:integration_{TEST_SUFFIX}",
            f"mrn:account:integration_{TEST_SUFFIX}"
        )
        
        if test_domain:
            domain_id = test_domain['id']
            CREATED_TEST_DATA.append({'type': 'domain', 'id': domain_id})
            print(f"✅ Domain creado: {domain_id}")
            
            # Crear portfolio con domain
            portfolio_service = PortfolioService()
            portfolio_id = portfolio_service.create_portfolio_with_domains(
                f"INTEGRATION_PORTFOLIO_{TEST_SUFFIX}",
                [domain_id]
            )
            
            if portfolio_id:
                CREATED_TEST_DATA.append({'type': 'portfolio', 'id': portfolio_id})
                print(f"✅ Portfolio creado: {portfolio_id}")
                
                # Crear eventos para el domain
                uow = SQLAlchemyUnitOfWork()
                event_service = EventService(uow)
                
                events_created = []
                for i in range(3):
                    timestamp = int((datetime.now() + timedelta(minutes=i)).timestamp() * 1000)
                    event = event_service.create_event(
                        domain_id, -34.6118 + (i * 0.001), -58.3960 + (i * 0.001), 
                        50 + i, f"integration_test_{i}", timestamp, 100 + i, 90 + i
                    )
                    if event:
                        events_created.append(event['id'])
                        CREATED_TEST_DATA.append({'type': 'event', 'id': event['id']})
                
                if len(events_created) == 3:
                    print(f"✅ {len(events_created)} eventos creados")
                    
                    # Verificar eventos por domain
                    retrieved_events = event_service.get_events_by_domain(domain_id)
                    if retrieved_events and len(retrieved_events) >= 3:
                        print(f"✅ Eventos recuperados: {len(retrieved_events)}")
                        integration_results['domain_portfolio_events'] = {
                            "status": "✅ OK",
                            "domain_id": domain_id,
                            "portfolio_id": portfolio_id,
                            "events_count": len(retrieved_events)
                        }
                    else:
                        integration_results['domain_portfolio_events'] = {
                            "status": "⚠️ PARTIAL", "issue": "Event retrieval issue"
                        }
                else:
                    integration_results['domain_portfolio_events'] = {
                        "status": "❌ FAIL", "issue": "Event creation incomplete"
                    }
            else:
                integration_results['domain_portfolio_events'] = {
                    "status": "❌ FAIL", "issue": "Portfolio creation failed"
                }
        else:
            integration_results['domain_portfolio_events'] = {
                "status": "❌ FAIL", "issue": "Domain creation failed"
            }
            
    except Exception as e:
        print(f"❌ Error en integración domain-portfolio-events: {e}")
        integration_results['domain_portfolio_events'] = {"status": "❌ FAIL", "error": str(e)}
    
    # 2. Flujo Security: User -> Role -> Permission
    print("\n🔐 2. FLUJO USER -> ROLE -> PERMISSION")
    try:
        from security.services.user_service import UserService
        from security.services.access_control_service import AccessControlService
        
        user_service = UserService()
        access_service = AccessControlService()
        
        # Crear usuario
        integration_user = user_service.register_user(
            f"integration_user_{TEST_SUFFIX}",
            f"integration_{TEST_SUFFIX}@test.com",
            "integration_pass_123",
            f"Integration User {TEST_SUFFIX}"
        )
        
        if integration_user:
            user_id = integration_user.id
            CREATED_TEST_DATA.append({'type': 'user', 'id': user_id})
            print(f"✅ Integration user creado: {user_id}")
            
            # Crear permission
            integration_permission = access_service.create_permission(
                f"integration_permission_{TEST_SUFFIX}",
                "Integration test permission"
            )
            
            if integration_permission:
                permission_id = integration_permission.id
                CREATED_TEST_DATA.append({'type': 'permission', 'id': permission_id})
                print(f"✅ Integration permission creado: {permission_id}")
                
                # Crear role
                integration_role = access_service.create_role(
                    f"integration_role_{TEST_SUFFIX}",
                    "Integration test role"
                )
                
                if integration_role:
                    role_id = integration_role.id
                    CREATED_TEST_DATA.append({'type': 'role', 'id': role_id})
                    print(f"✅ Integration role creado: {role_id}")
                    
                    # Asignar permission a role
                    permission_assigned = access_service.assign_permission_to_role(
                        role_id, permission_id
                    )
                    
                    if permission_assigned:
                        print("✅ Permission asignado a role")
                        
                        # Asignar role a user
                        role_assigned = access_service.assign_role_to_user(
                            user_id, role_id
                        )
                        
                        if role_assigned:
                            print("✅ Role asignado a user")
                            integration_results['user_role_permission'] = {
                                "status": "✅ OK",
                                "user_id": user_id,
                                "role_id": role_id,
                                "permission_id": permission_id
                            }
                        else:
                            integration_results['user_role_permission'] = {
                                "status": "❌ FAIL", "issue": "Role assignment failed"
                            }
                    else:
                        integration_results['user_role_permission'] = {
                            "status": "❌ FAIL", "issue": "Permission assignment failed"
                        }
                else:
                    integration_results['user_role_permission'] = {
                        "status": "❌ FAIL", "issue": "Role creation failed"
                    }
            else:
                integration_results['user_role_permission'] = {
                    "status": "❌ FAIL", "issue": "Permission creation failed"
                }
        else:
            integration_results['user_role_permission'] = {
                "status": "❌ FAIL", "issue": "User creation failed"
            }
            
    except Exception as e:
        print(f"❌ Error en integración user-role-permission: {e}")
        integration_results['user_role_permission'] = {"status": "❌ FAIL", "error": str(e)}
    
    return integration_results

# Ejecutar tests de integración
integration_test_results = test_integration_scenarios()
TEST_RESULTS['integration_scenarios'] = integration_test_results

print("\n📋 Resumen Integration Scenarios:")
for scenario, result in integration_test_results.items():
    print(f"  {scenario}: {result['status']}")

## 📊 8. Análisis de Resultados y Reporte Final

In [ ]:
def generate_comprehensive_report():
    """Genera reporte completo de todos los tests"""
    print("📊 REPORTE COMPLETO DE TESTING - SISTEMA STRIX")
    print("=" * 60)
    
    # Contadores generales
    total_tests = 0
    successful_tests = 0
    partial_tests = 0
    failed_tests = 0
    
    def count_results(results_dict):
        nonlocal total_tests, successful_tests, partial_tests, failed_tests
        for key, result in results_dict.items():
            if isinstance(result, dict) and 'status' in result:
                total_tests += 1
                if "✅" in result['status']:
                    successful_tests += 1
                elif "⚠️" in result['status']:
                    partial_tests += 1
                elif "❌" in result['status']:
                    failed_tests += 1
    
    # Análisis por categoría
    print("\n🔍 ANÁLISIS POR CATEGORÍA:")
    print("-" * 40)
    
    # 1. Imports
    print("\n📦 1. IMPORTS:")
    if 'imports' in TEST_RESULTS:
        import_success = sum(1 for r in TEST_RESULTS['imports'].values() if "✅" in r['status'])
        import_total = len(TEST_RESULTS['imports'])
        print(f"   Exitosos: {import_success}/{import_total} ({(import_success/import_total*100):.1f}%)")
        total_tests += import_total
        successful_tests += import_success
        failed_tests += (import_total - import_success)
        
        # Mostrar imports fallidos
        failed_imports = [name for name, result in TEST_RESULTS['imports'].items() if "❌" in result['status']]
        if failed_imports:
            print(f"   ❌ Fallidos: {', '.join(failed_imports)}")
    
    # 2. Health Check
    print("\n🏥 2. HEALTH CHECK:")
    if 'health_check' in TEST_RESULTS:
        health_success = sum(1 for r in TEST_RESULTS['health_check'].values() if "✅" in r['status'])
        health_total = len(TEST_RESULTS['health_check'])
        print(f"   Servicios OK: {health_success}/{health_total}")
        for service, result in TEST_RESULTS['health_check'].items():
            print(f"     {service}: {result['status']}")
        count_results(TEST_RESULTS['health_check'])
    
    # 3. Core Services
    print("\n🧱 3. CORE SERVICES:")
    if 'core_services' in TEST_RESULTS:
        for service, result in TEST_RESULTS['core_services'].items():
            print(f"   {service}: {result['status']}")
        count_results(TEST_RESULTS['core_services'])
    
    # 4. Security Services
    print("\n🔐 4. SECURITY SERVICES:")
    if 'security_services' in TEST_RESULTS:
        for service, result in TEST_RESULTS['security_services'].items():
            print(f"   {service}: {result['status']}")
        count_results(TEST_RESULTS['security_services'])
    
    # 5. API Endpoints
    print("\n🌐 5. API ENDPOINTS:")
    if 'api_endpoints' in TEST_RESULTS:
        for endpoint, result in TEST_RESULTS['api_endpoints'].items():
            print(f"   {endpoint}: {result['status']}")
        count_results(TEST_RESULTS['api_endpoints'])
    
    # 6. Integration Scenarios
    print("\n🔗 6. INTEGRATION SCENARIOS:")
    if 'integration_scenarios' in TEST_RESULTS:
        for scenario, result in TEST_RESULTS['integration_scenarios'].items():
            print(f"   {scenario}: {result['status']}")
        count_results(TEST_RESULTS['integration_scenarios'])
    
    # Estadísticas generales
    success_rate = (successful_tests / total_tests * 100) if total_tests > 0 else 0
    partial_rate = (partial_tests / total_tests * 100) if total_tests > 0 else 0
    fail_rate = (failed_tests / total_tests * 100) if total_tests > 0 else 0
    
    print("\n📈 ESTADÍSTICAS GENERALES:")
    print("=" * 40)
    print(f"Total de tests ejecutados: {total_tests}")
    print(f"✅ Exitosos: {successful_tests} ({success_rate:.1f}%)")
    print(f"⚠️ Parciales: {partial_tests} ({partial_rate:.1f}%)")
    print(f"❌ Fallidos: {failed_tests} ({fail_rate:.1f}%)")
    
    # Interpretación de resultados
    print("\n🎯 INTERPRETACIÓN:")
    print("-" * 30)
    
    if success_rate >= 90:
        print("🎉 EXCELENTE: El sistema está en muy buen estado")
        print("   ✅ Todos los componentes principales funcionan correctamente")
        print("   ✅ Listo para desarrollo y testing avanzado")
    elif success_rate >= 70:
        print("👍 BUENO: El sistema funciona bien con algunos problemas menores")
        print("   ✅ Componentes principales operativos")
        print("   ⚠️ Revisar componentes con estado parcial")
    elif success_rate >= 50:
        print("⚠️ REGULAR: El sistema funciona pero requiere atención")
        print("   🔧 Varios componentes necesitan configuración")
        print("   📖 Revisar documentación de setup")
    else:
        print("🚨 CRÍTICO: El sistema requiere atención inmediata")
        print("   🔧 Problemas fundamentales en la configuración")
        print("   📞 Consultar con el equipo de desarrollo")
    
    # Recomendaciones específicas
    print("\n💡 RECOMENDACIONES:")
    print("-" * 25)
    
    if 'health_check' in TEST_RESULTS:
        if any("❌" in r['status'] for r in TEST_RESULTS['health_check'].values()):
            print("🔧 Configurar conexiones a base de datos y/o S3")
    
    if failed_tests > 0:
        print("📋 Revisar logs de errores para componentes fallidos")
        print("🔄 Re-ejecutar tests después de corregir configuración")
    
    if partial_tests > 0:
        print("⚠️ Investigar componentes con estado parcial")
        print("📊 Considerar si los warnings son críticos para tu uso")
    
    if successful_tests > 0:
        print("✅ Usar componentes exitosos para desarrollo continuo")
    
    # Información de datos de prueba
    print("\n🗑️ DATOS DE PRUEBA CREADOS:")
    print("-" * 30)
    if CREATED_TEST_DATA:
        data_summary = {}
        for item in CREATED_TEST_DATA:
            data_type = item['type']
            if data_type not in data_summary:
                data_summary[data_type] = []
            data_summary[data_type].append(item['id'])
        
        for data_type, ids in data_summary.items():
            print(f"   {data_type}: {len(ids)} registros creados")
        
        print("\n⚠️ NOTA: Los datos de prueba permanecen en la base de datos.")
        print("   Ejecutar limpieza manual si es necesario.")
    else:
        print("   No se crearon datos de prueba")
    
    # Timestamp del reporte
    print(f"\n🕐 Reporte generado: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"🏷️ Test session: {TEST_SUFFIX}")
    
    return {
        'total_tests': total_tests,
        'successful_tests': successful_tests,
        'partial_tests': partial_tests,
        'failed_tests': failed_tests,
        'success_rate': success_rate,
        'test_data_created': len(CREATED_TEST_DATA),
        'timestamp': datetime.now().isoformat()
    }

# Generar reporte final
final_report = generate_comprehensive_report()

## 🧹 9. Limpieza de Datos de Prueba (Opcional)

In [ ]:
def cleanup_test_data():
    """Limpia los datos de prueba creados durante el testing"""
    print("🧹 Iniciando limpieza de datos de prueba...")
    cleanup_results = {}
    
    if not CREATED_TEST_DATA:
        print("✅ No hay datos de prueba para limpiar")
        return {"status": "✅ OK", "message": "No test data to clean"}
    
    try:
        # Importar servicios necesarios
        from core.services.domain_service import DomainService
        from core.services.portfolio_service import PortfolioService
        from core.services.event_service import EventService
        from core.infraestructure.uow.sqlalchemy_uow import SQLAlchemyUnitOfWork
        from security.services.user_service import UserService
        from security.services.access_control_service import AccessControlService
        
        # Organizar datos por tipo
        data_by_type = {}
        for item in CREATED_TEST_DATA:
            data_type = item['type']
            if data_type not in data_by_type:
                data_by_type[data_type] = []
            data_by_type[data_type].append(item['id'])
        
        print(f"📋 Datos a limpiar: {data_by_type}")
        
        # Limpiar en orden específico (dependencias)
        cleanup_order = ['event', 'portfolio', 'domain', 'user', 'role', 'permission']
        
        for data_type in cleanup_order:
            if data_type in data_by_type:
                ids = data_by_type[data_type]
                print(f"\n🗑️ Limpiando {data_type}s: {ids}")
                
                cleaned_count = 0
                
                for item_id in ids:
                    try:
                        if data_type == 'domain':
                            domain_service = DomainService()
                            if domain_service.delete_domain(item_id):
                                cleaned_count += 1
                                
                        elif data_type == 'portfolio':
                            portfolio_service = PortfolioService()
                            if portfolio_service.delete_portfolio(item_id):
                                cleaned_count += 1
                                
                        elif data_type == 'event':
                            uow = SQLAlchemyUnitOfWork()
                            event_service = EventService(uow)
                            if event_service.delete_event(item_id):
                                cleaned_count += 1
                                
                        # Nota: Users, roles y permissions generalmente no se eliminan
                        # en producción, solo en testing
                        elif data_type in ['user', 'role', 'permission']:
                            print(f"   ⚠️ {data_type} {item_id} no eliminado (política de seguridad)")
                            
                    except Exception as e:
                        print(f"   ❌ Error eliminando {data_type} {item_id}: {e}")
                
                print(f"   ✅ {cleaned_count}/{len(ids)} {data_type}s eliminados")
                cleanup_results[data_type] = {
                    "attempted": len(ids),
                    "cleaned": cleaned_count
                }
        
        total_cleaned = sum(r.get('cleaned', 0) for r in cleanup_results.values())
        total_attempted = sum(r.get('attempted', 0) for r in cleanup_results.values())
        
        print(f"\n📊 Resumen limpieza: {total_cleaned}/{total_attempted} elementos limpiados")
        
        return {
            "status": "✅ OK",
            "total_cleaned": total_cleaned,
            "total_attempted": total_attempted,
            "details": cleanup_results
        }
        
    except Exception as e:
        print(f"❌ Error general en limpieza: {e}")
        return {"status": "❌ FAIL", "error": str(e)}

# Para ejecutar la limpieza, descomenta la siguiente línea:
# cleanup_result = cleanup_test_data()

print("⚠️ LIMPIEZA DE DATOS DESHABILITADA")
print("Para limpiar los datos de prueba, descomenta la línea anterior")
print(f"Datos creados durante esta sesión: {len(CREATED_TEST_DATA)} elementos")

## 💾 10. Exportar Resultados

In [ ]:
def export_test_results():
    """Exporta los resultados a diferentes formatos"""
    print("💾 Exportando resultados...")
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # 1. JSON completo
    json_data = {
        "test_session": TEST_SUFFIX,
        "timestamp": datetime.now().isoformat(),
        "summary": final_report,
        "detailed_results": TEST_RESULTS,
        "test_data_created": CREATED_TEST_DATA
    }
    
    json_filename = f"strix_test_results_{timestamp}.json"
    
    try:
        with open(json_filename, 'w', encoding='utf-8') as f:
            json.dump(json_data, f, indent=2, ensure_ascii=False, default=str)
        print(f"✅ Resultados JSON guardados: {json_filename}")
    except Exception as e:
        print(f"❌ Error guardando JSON: {e}")
    
    # 2. Resumen en CSV
    csv_data = []
    
    for category, results in TEST_RESULTS.items():
        if isinstance(results, dict):
            for test_name, result in results.items():
                if isinstance(result, dict) and 'status' in result:
                    csv_data.append({
                        'category': category,
                        'test_name': test_name,
                        'status': result['status'],
                        'details': result.get('details', result.get('error', result.get('issue', '')))
                    })
    
    if csv_data:
        try:
            df = pd.DataFrame(csv_data)
            csv_filename = f"strix_test_summary_{timestamp}.csv"
            df.to_csv(csv_filename, index=False, encoding='utf-8')
            print(f"✅ Resumen CSV guardado: {csv_filename}")
        except Exception as e:
            print(f"❌ Error guardando CSV: {e}")
    
    # 3. Reporte de texto
    txt_filename = f"strix_test_report_{timestamp}.txt"
    
    try:
        with open(txt_filename, 'w', encoding='utf-8') as f:
            f.write(f"REPORTE DE TESTING SISTEMA STRIX\n")
            f.write(f"="*50 + "\n\n")
            f.write(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"Test Session: {TEST_SUFFIX}\n\n")
            
            f.write(f"RESUMEN GENERAL:\n")
            f.write(f"Total tests: {final_report['total_tests']}\n")
            f.write(f"Exitosos: {final_report['successful_tests']} ({final_report['success_rate']:.1f}%)\n")
            f.write(f"Parciales: {final_report['partial_tests']}\n")
            f.write(f"Fallidos: {final_report['failed_tests']}\n\n")
            
            for category, results in TEST_RESULTS.items():
                f.write(f"{category.upper()}:\n")
                f.write("-" * 30 + "\n")
                if isinstance(results, dict):
                    for test_name, result in results.items():
                        if isinstance(result, dict) and 'status' in result:
                            f.write(f"  {test_name}: {result['status']}\n")
                f.write("\n")
        
        print(f"✅ Reporte TXT guardado: {txt_filename}")
    except Exception as e:
        print(f"❌ Error guardando TXT: {e}")
    
    print(f"\n📁 Archivos generados en el directorio actual:")
    print(f"   📄 {json_filename} (datos completos)")
    if csv_data:
        print(f"   📊 {csv_filename} (resumen tabular)")
    print(f"   📝 {txt_filename} (reporte legible)")

# Exportar resultados
export_test_results()

print("\n🎉 TESTING COMPLETO")
print(f"Sesión de testing: {TEST_SUFFIX}")
print(f"Elementos de datos creados: {len(CREATED_TEST_DATA)}")
print(f"Tiempo total: {datetime.now().strftime('%H:%M:%S')}")